In [43]:
import pandas as pd
import numpy as np
import math
import requests
from bs4 import BeautifulSoup as soup
import re

In [2]:
cricsheet_df=pd.read_csv('./data/all_matches.csv')

/Users/kishan/Desktop/IPL/.ipl/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
def phase_cat(over_number):
    '''Determine phase'''
    if over_number<=6:
        phase="PowerPlay"
    if (over_number>6) & (over_number<=15):
        phase="Middle overs"
    if over_number>15:
        phase="Death overs"
    return phase

In [4]:
cricsheet_df.drop(columns=['other_wicket_type','other_player_dismissed'],axis=1,inplace=True)
cricsheet_df['start_date']=pd.to_datetime(cricsheet_df['start_date'])
print(cricsheet_df.loc[cricsheet_df['start_date']>='2018','start_date'].describe(include='all'))
cdf=cricsheet_df.loc[cricsheet_df['start_date']>='2018'].reset_index(drop=True)
cdf=cdf.loc[cdf['innings']<=2]
cdf.loc[cdf['batting_team']=='Delhi Daredevils','batting_team']='Delhi Capitals'
cdf.loc[cdf['bowling_team']=='Delhi Daredevils','bowling_team']='Delhi Capitals'
cdf.loc[cdf['batting_team']=='Kings XI Punjab','batting_team']='Punjab Kings'
cdf.loc[cdf['bowling_team']=='Kings XI Punjab','bowling_team']='Punjab Kings'
cdf.drop(columns=['wides','noballs','byes','legbyes','penalty'],inplace=True)
cdf['total_runs']=cdf['runs_off_bat']+cdf['extras']
##Get over number
cdf['over_number']=cdf['ball'].apply(lambda x:int(math.ceil(x)))


count                   47289
unique                    161
top       2020-10-18 00:00:00
freq                      520
first     2018-04-07 00:00:00
last      2021-04-23 00:00:00
Name: start_date, dtype: object


<ipython-input-4-522116867acb>:3: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  print(cricsheet_df.loc[cricsheet_df['start_date']>='2018','start_date'].describe(include='all'))


In [5]:
cdf['phase']=cdf['over_number'].apply(phase_cat)

In [6]:
cdf['wicket_fallen']=cdf['wicket_type'].apply(lambda x:0 if pd.isnull(x) else 1)

In [7]:
cdf['pair_at_crease']=cdf.apply(lambda x:x['striker']+'-'+x['non_striker'],axis=1)

In [8]:
cdf['batting_pair']=cdf['pair_at_crease'].apply(lambda x:list(set(x.split('-'))))

In [9]:
cdf['batting_pair']=cdf['batting_pair'].apply(lambda x:str(x[0]+'-'+x[1]) if len(x)>=2 else x)

### Update player database

In [10]:
ipl_players=pd.DataFrame(cdf.loc[:,['striker','non_striker','bowler']].drop_duplicates().melt()['value'].drop_duplicates().tolist(),columns=['ipl_players'])

In [89]:
ipl_players

,ipl_players
0,V Kohli
1,PA Patel
2,MM Ali
3,AB de Villiers
4,SO Hetmyer
...,...
238,Monu Kumar
239,AF Milne
240,LI Meriwala
241,Jalaj S Saxena


In [25]:
from azure.storage.blob import BlobServiceClient

In [26]:
connect_str='DefaultEndpointsProtocol=https;AccountName=kishcricket21;AccountKey=wYdGh/fMOb2B/P2RmHPqwPGBYy65c+jpvw63hcNZNHwC03bNJNrYrh8+NhjB7pQgCaq84QSdrE27PbIN9uF1wQ==;EndpointSuffix=core.windows.net'

In [27]:
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

In [28]:
list(blob_service_client.list_containers())

[{'name': 'playerdb', 'last_modified': datetime.datetime(2021, 4, 24, 17, 7, 58, tzinfo=datetime.timezone.utc), 'etag': '"0x8D9074382E36D3A"', 'lease': {'status': 'unlocked', 'state': 'available', 'duration': None}, 'public_access': 'container', 'has_immutability_policy': False, 'deleted': None, 'version': None, 'has_legal_hold': False, 'metadata': None, 'encryption_scope': <azure.storage.blob._models.ContainerEncryptionScope object at 0x7f8bfaa741c0>}]

In [29]:
blob_client = blob_service_client.get_blob_client(container='playerdb',blob='players_formatted.xlsx')

In [30]:
with open('./data2/players_db.xlsx', "wb") as download_file:
    download_file.write(blob_client.download_blob().readall())

In [93]:
player_df=pd.read_excel('./data2/players_db.xlsx')

In [94]:
pdf=player_df.loc[player_df['cricsheet_name'].isin(ipl_players['ipl_players'])]['cricsheet_name'].tolist()

In [95]:
players_to_add=[i for i in ipl_players['ipl_players'] if i not in pdf]
players_to_add

['CH Gayle',
 'M Shahrukh Khan',
 'JA Richardson',
 'Lalit Yadav',
 'Virat Singh',
 'C Sakariya',
 'RP Meredith',
 'LI Meriwala',
 'Jalaj S Saxena']

In [98]:
google_url=f"https://www.google.com/search?q="

In [96]:
##Build a crawler to compile player data from wikipedia

def crawl_player(player):
    '''Using a string of the player name, crawl to google and hit the relevant Wiki URL, to store the player attributes and value table
    Returns: Dataframe of attributes and values for a particular player'''
    
    ##Get google results
    google_results=requests.get(google_url,params={'q':player+"cricket wiki"})

    ##Format the google result into soup, so that it can be parsed for the relevant URLs
    google_soup=soup(google_results.text,'html.parser')

    try:
        wiki_url=[links['href'].split('&')[0].split('?q=') for links in google_soup.find_all('a') 
        if 'en.wikipedia.org/wiki' in links['href']][0][1].strip()
    except Exception as ex:
        wiki_url=''
    #print(wiki_url)
    ##If there is no relevant wiki url found
    if len(wiki_url)==0:
        crawl_result_df=pd.DataFrame({'cricsheet_name':[player],'Result':'No relevant wiki URL found for company from google crawler'},index=[0])
        wiki_requests=''
    else:
        ##Access the relevant player wiki URL
        wiki_requests=requests.get(wiki_url)

        ##Format wiki page content into a soup
    try:
        wiki_soup=soup(wiki_requests.text,'html.parser')
    except Exception as ex:
        wiki_soup=''

    ##Get the relevant table element from web page

    ##Get the first table element for the page

    try:
        ##Get the personal information table for the player

        right_table=[tbl for tbl in wiki_soup.find_all('table',{'class':'infobox vcard'})][0]

    except Exception as ex:
        right_table=''
        crawl_result_df=pd.DataFrame({'cricsheet_name':[player],'Result':'wiki url not parsed correctly'},index=[0])

    ##Handle exception if ther eare no rows found

    try:
        ##Parse all rows
        rows=right_table.find_all('tr')
        ##Initialize batting and bowling style attributes
        bat_style,bowl_style,full_name='','',''
        for r in rows:
            ##If row exists with a header, i.e. if there is an attribute then store the value
            try:
                rowtext=r.find('th').text
            except Exception as ex:
                rowtext=''
                pass

            if rowtext=='Bowling':
                bowl_style=r.find('td').text
            if rowtext=='Batting':
                bat_style=r.find('td').text
            if "Full" in rowtext:
                full_name=r.find('td').text

        crawl_result_df=pd.DataFrame({'cricsheet_name':[player],'battingStyle':[bat_style],'bowlingStyle':[bowl_style],'fullName':[full_name]})


    except Exception as ex:
        crawl_result_df=pd.DataFrame({'cricsheet_name':[player],'Result':'attributes values not parsed correctly'},index=[0])

    return crawl_result_df


In [99]:
added_df=pd.DataFrame()

for player in players_to_add:
    print(player)
    df=crawl_player(player)
    added_df=pd.concat([added_df,df])

CH Gayle
M Shahrukh Khan
JA Richardson
Lalit Yadav
Virat Singh
C Sakariya
RP Meredith
LI Meriwala
Jalaj S Saxena


In [100]:
def format_batting_styles(bat):
    bat=str(bat)
    if bat=='':
        batstyle=''
    if bat=='nan':
        batstyle='nan'
    if 'Right' in bat:
        batstyle='Right-hand bat'
    if 'Left' in bat:
        batstyle='Left-hand bat'
    return batstyle

In [101]:
added_df['battingStyle']=added_df['battingStyle'].apply(format_batting_styles)

In [109]:
##Do manual tagging
added_df.to_excel('./data2/added_df.xlsx',index=False)

In [103]:
player_df=pd.concat([player_df,added_df])

In [107]:
player_df.to_excel('./data2/players_db.xlsx')

In [111]:
pdf=player_df.loc[:,['cricsheet_name','battingStyle','bowlingStyle']].drop_duplicates()

pdf

,cricsheet_name,battingStyle,bowlingStyle
0,Mohammad Naveed,UAE1,Right-arm medium
1,Amjad Javed,UAE2,Right-arm medium
2,Kamran Shazad,UAE3,Right-arm medium-fast
3,Mohammad Tauqir,UAE4,Off-break
4,Fahad Alhashmi,UAE5,Right-arm fast-medium
...,...,...,...
0,Virat Singh,Left-hand bat,Right-arm leg spin
0,C Sakariya,Left-hand bat,Left-arm medium-fast
0,RP Meredith,Right-hand bat,Right-arm fast
0,LI Meriwala,Left-hand bat,Left-arm fast-medium


In [112]:
cdf=cdf.merge(pdf.loc[:,['cricsheet_name','battingStyle']],left_on='striker',right_on='cricsheet_name')

In [113]:
cdf=cdf.merge(pdf.loc[:,['cricsheet_name','bowlingStyle']],left_on='bowler',right_on='cricsheet_name')

In [114]:
cdf=cdf.loc[:,['match_id','season','start_date','venue','innings','ball','over_number','batting_team','bowling_team','wicket_type','runs_off_bat','extras','total_runs','phase','wicket_fallen','pair_at_crease','batting_pair','striker','battingStyle','bowler','bowlingStyle']]

#### Identify day games

In [280]:
cdf_daydf=cdf.loc[:,['match_id','start_date']].drop_duplicates()
cdf_daydf['m']=1
cdf_daydf.sort_values(by='start_date',inplace=True)
cdf2=cdf_daydf.groupby('start_date').rank()
cdf_daydf['timing']='night'
cdf_daydf.loc[(cdf2['match_id']==1.0)&(cdf2['m']==1.5),'timing']='day'
cdf=cdf.merge(cdf_daydf.loc[:,['match_id','timing']],on='match_id')

In [294]:
cdf.to_excel('./data2/ipl2021.xlsx',index=False)

In [118]:
espn_url='https://search.espncricinfo.com/ci/content/site/search.html?search={};type=player'

In [119]:
espn_search_url=espn_url.format(player)

In [120]:
espn_search_url

'https://search.espncricinfo.com/ci/content/site/search.html?search=Jalaj S Saxena;type=player'

In [64]:
search_page=requests.get(espn_search_url)

In [66]:
search_soup=soup(search_page.text,'html.parser')

In [70]:
[a['href'] for a in search_soup.find_all('a') if 'content/player' in a['href']]

['https://www.espncricinfo.com/ci/content/player/index.html',
 '/southafrica/content/player/545467.html']

In [38]:
player_url='https://www.espncricinfo.com/player/junior-dala-545467'

In [40]:
r=requests.get(player_url)

In [44]:
profile_soup=soup(r.text,'html.parser')

In [48]:
profile_texts=['Full Name','Born','Age','Batting Style','Bowling Style']

In [47]:
profile_soup.find_all('p',{'class':'text-uppercase gray-700 mb-0 pb-0-5 player-card-heading'})

[<p class="text-uppercase gray-700 mb-0 pb-0-5 player-card-heading">Full Name</p>,
 <p class="text-uppercase gray-700 mb-0 pb-0-5 player-card-heading">Born</p>,
 <p class="text-uppercase gray-700 mb-0 pb-0-5 player-card-heading">Age</p>,
 <p class="text-uppercase gray-700 mb-0 pb-0-5 player-card-heading">Batting Style</p>,
 <p class="text-uppercase gray-700 mb-0 pb-0-5 player-card-heading">Bowling Style</p>,
 <p class="text-uppercase gray-700 mb-0 pb-0-5 player-card-heading">Playing Role</p>]

In [50]:
[p.text for p in profile_soup.find_all('h5',{'class':'player-card-description gray-900'})]

['Carl Junior Dala',
 'December 29, 1989, Lusaka, Zambia',
 '31y 116d',
 'Right hand bat',
 'Right arm medium',
 'Bowler']